In [ ]:
# Pyton Package Installation

## virtual environment setup
%pip install conda

## nlp libraries
%pip install --upgrade gensim
%pip install top2vec

## machine learning libraries
%pip install transformers
%pip install torch

## visualization libraries
%pip install matplotlib
%pip install seaborn
%pip install pyldavis
%pip install cartopy

## datahandling libraries
%pip install pandas

In [ ]:
# NLP-Pipeline
import nltk 
import sklearn
import gensim
# import spaCy as sp
import re

# ML
import transformers
import torch 

## Embeddings

import top2vec as t2v 

## Visualisierung
import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud as wc
import pyLDAvis
import cartopy as ctp

# Sortieren
import venv
import numpy as np

TF-IDF

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Import des Datensatzes
complaints = [complaints_data_cleanced.csv]
    "I am very unhappy with the service I received.",
    "The product quality is excellent and exceeded my expectations.",
    "Customer support was unhelpful and rude.",
    "Delivery was prompt and the packaging was secure.",
    "I had a terrible experience with the billing department."
]
#TF-IDF + LDA

vectorizer = TfidfVectorizer(max_df=0.6, min_df=1, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(complaints)
lda = LatentDirichletAllocation(n_components=10)
lda_topics = lda.fit_transform(tfidf_matrix)

print(lda_topics)

[[0.6705771  0.03660255 0.03660254 0.03660255 0.03660255 0.03660254
  0.03660255 0.03660254 0.03660255 0.03660255]
 [0.72188466 0.03090171 0.0309017  0.03090171 0.03090171 0.0309017
  0.03090171 0.0309017  0.03090171 0.03090171]
 [0.03333333 0.03333334 0.69999997 0.03333334 0.03333334 0.03333334
  0.03333334 0.03333334 0.03333334 0.03333334]
 [0.03333333 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334
  0.03333334 0.69999997 0.03333334 0.03333334]
 [0.03333333 0.03333334 0.03333334 0.03333334 0.03333334 0.69999997
  0.03333334 0.03333334 0.03333334 0.03333334]]
